In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [2]:
b = np.fromfile('tensors_2010.dat',dtype=np.float64).reshape((8000,-1,200))
Y = np.fromfile('tensors_2010_labels.dat',dtype=np.int)
position_tag_1 = np.fromfile('tensors_2010_entity1.dat',dtype=np.int)
position_tag_2 = np.fromfile('tensors_2010_entity2.dat',dtype=np.int)
position_tag = np.concatenate((position_tag_1,position_tag_2)).reshape((8000,-1))
b.shape,Y.shape,position_tag_1.shape,position_tag_2.shape,position_tag.shape

((8000, 85, 200), (8000,), (8000,), (8000,), (8000, 2))

In [3]:
inputs = []
for i in range(8000):
    inputs.append((torch.from_numpy(b[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [4]:
t_e = []
t_y = []
for E,Y in test:
    t_e.append(E)
    t_y.append(Y)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_e.shape, t_y.shape

(torch.Size([2400, 85, 200]), torch.Size([2400]))

# glove_rnn_max: 0.6341666666666667 15轮

In [5]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_max(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=64,out_features=10)
        self.maxpool = torch.nn.MaxPool2d((85,1))

    def forward(self,x):
        output,(h_n,c_n)=self.rnn(x.float())
        output_in_last_timestep = self.maxpool(output)
        output_in_last_timestep = output_in_last_timestep.reshape((output_in_last_timestep.shape[0],-1))
        x=self.out(output_in_last_timestep)
        return x

In [ ]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_max()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,301): 
    for step, (batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_max(
  (rnn): LSTM(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
  (maxpool): MaxPool2d(kernel_size=(85, 1), stride=(85, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.245 loss: 2.2047150135040283
2: accuracy:0.5195833333333333 loss: 1.2479912042617798
3: accuracy:0.5895833333333333 loss: 0.9295550584793091
4: accuracy:0.62125 loss: 0.7006310224533081
5: accuracy:0.62125 loss: 0.4652349054813385
6: accuracy:0.63 loss: 0.44489288330078125
7: accuracy:0.6033333333333334 loss: 0.23127657175064087
8: accuracy:0.60875 loss: 0.255601167678833
9: accuracy:0.6079166666666667 loss: 0.1630931794643402
10: accuracy:0.6304166666666666 loss: 0.07207967340946198
11: accuracy:0.615 loss: 0.060785531997680664
12: accuracy:0.62375 loss: 0.03939204290509224
13: accuracy:0.60875 loss: 0.07682640850543976
14: accuracy:0.6058333333333333 loss: 0.056524571031332016
15: accuracy:0.6341666666666667 loss: 0.022927533835172653
16: accuracy:0.619166666666

147: accuracy:0.6045833333333334 loss: 0.002690548077225685
148: accuracy:0.6020833333333333 loss: 0.00649191252887249
149: accuracy:0.60375 loss: 0.0020469296723604202
150: accuracy:0.6004166666666667 loss: 0.0021061208099126816
151: accuracy:0.6016666666666667 loss: 0.009070277214050293
152: accuracy:0.6058333333333333 loss: 0.0018863584846258163
153: accuracy:0.60125 loss: 0.0017761960625648499
154: accuracy:0.6041666666666666 loss: 0.0068917348980903625
155: accuracy:0.60375 loss: 0.0017234813421964645
156: accuracy:0.6025 loss: 0.0015098769217729568
157: accuracy:0.6045833333333334 loss: 0.0013207755982875824
158: accuracy:0.60375 loss: 0.00150214321911335
159: accuracy:0.6045833333333334 loss: 0.0013091228902339935
160: accuracy:0.6045833333333334 loss: 0.0011624302715063095
161: accuracy:0.605 loss: 0.0047631217166781425
162: accuracy:0.6033333333333334 loss: 0.0010217130184173584
163: accuracy:0.6045833333333334 loss: 0.0009601395577192307
164: accuracy:0.60375 loss: 0.00100787

# glove:0.44680851063829785

In [7]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN_old(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=64,out_features=10)

    def forward(self,x):
        output,(h_n,c_n)=self.rnn(x.float())
        output_in_last_timestep=h_n[-1,:,:]
        x=self.out(output_in_last_timestep)
        return x

In [8]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = RNN_old()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,251): 
    for step, (batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN_old(
  (rnn): LSTM(200, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)
1: accuracy:0.19858156028368795 loss: 2.2743892669677734
2: accuracy:0.1702127659574468 loss: 2.282547950744629
3: accuracy:0.1702127659574468 loss: 2.2899293899536133
4: accuracy:0.19858156028368795 loss: 2.2691707611083984
5: accuracy:0.19858156028368795 loss: 2.262784719467163
6: accuracy:0.19858156028368795 loss: 2.256842851638794
7: accuracy:0.19858156028368795 loss: 2.2975029945373535
8: accuracy:0.19858156028368795 loss: 2.277923345565796
9: accuracy:0.19858156028368795 loss: 2.2180051803588867
10: accuracy:0.19858156028368795 loss: 2.230205535888672
11: accuracy:0.1702127659574468 loss: 2.2795324325561523
12: accuracy:0.1702127659574468 loss: 2.2521166801452637
13: accuracy:0.1702127659574468 loss: 2.2433652877807617
14: accuracy:0.1702127659574468 loss: 2.273130416870117
15: accuracy:0.19858156028368795 loss: 2.2797694206237793
16: accuracy:0.19858156028368795 loss: 

143: accuracy:0.19858156028368795 loss: 2.2842042446136475
144: accuracy:0.19858156028368795 loss: 2.254070281982422
145: accuracy:0.19858156028368795 loss: 2.2810349464416504
146: accuracy:0.19858156028368795 loss: 2.2747700214385986
147: accuracy:0.19858156028368795 loss: 2.3059682846069336
148: accuracy:0.19858156028368795 loss: 2.229618787765503
149: accuracy:0.19858156028368795 loss: 2.288832902908325
150: accuracy:0.19858156028368795 loss: 2.2772305011749268
151: accuracy:0.1702127659574468 loss: 2.238725185394287
152: accuracy:0.1702127659574468 loss: 2.285245895385742
153: accuracy:0.1702127659574468 loss: 2.2971010208129883
154: accuracy:0.1702127659574468 loss: 2.2709646224975586
155: accuracy:0.1702127659574468 loss: 2.2439606189727783
156: accuracy:0.1702127659574468 loss: 2.2653632164001465
157: accuracy:0.1702127659574468 loss: 2.27630615234375
158: accuracy:0.1702127659574468 loss: 2.2898292541503906
159: accuracy:0.1702127659574468 loss: 2.288764715194702
160: accuracy:

# N-new 0.6879432624113475 112轮

In [3]:
inputs = []
MAX_LENGTH = 53
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=1)
    a = np.log(a)
    if a.shape[0] < MAX_LENGTH:
        add = MAX_LENGTH - a.shape[0]
        adds = np.zeros(add)
        a = np.concatenate((a, adds))
    inputs.append(a)

In [4]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation

In [5]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))

In [6]:
b.shape,use.shape

((467, 53, 200), (467, 53))

In [7]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [8]:
t_s = []
t_e = []
t_y = []
for E,S,Y in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape

(torch.Size([141, 53]), torch.Size([141, 53, 200]), torch.Size([141]))

In [98]:
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(
            input_size = 200,       # rnn input
            hidden_size = 64,     # rnn hidden unit
            num_layers = 1,       # 有几层 RNN layers
            batch_first=True,   # input & output 会是以 batch size 为第一维度的特征集 e.g. (batch, time_step, input_size)
        )
        self.out=torch.nn.Linear(in_features=65 ,out_features=7)
        self.maxpool = torch.nn.MaxPool2d((53,1))

    def forward(self,sen,x):
        output,(h_n,c_n)=self.rnn(x.float())
        sen = sen.reshape((sen.shape[0],sen.shape[1],1))  # (BATCH_SIZE,53,1)
        output1 = torch.bmm(torch.transpose(output,1,2).float(),sen.float())  # (BATCH_SIZE,64,1)
        output1 = torch.sigmoid(output1)
        output1 = torch.softmax(output1,1)
        output1 = torch.bmm(output,output1) # (BATCH_SIZE,53,1)
        output = torch.cat((output1,output),2) # (BATCH_SIZE,53,65)
        output = self.maxpool(output)   # (BATCH_SIZE,1,65)
        output = output.reshape((output.shape[0],-1)) # (BATCH_SIZE,7)
        x1 = self.out(output.float())
        return x1

In [99]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
    drop_last=True,
)
net = RNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y) in enumerate(loader):
        pred = net(sen, batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 1: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s, t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss}")

RNN(
  (rnn): LSTM(200, 64, batch_first=True)
  (out): Linear(in_features=65, out_features=7, bias=True)
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
)
1: accuracy:0.3049645390070922 loss: 1.9433010816574097
2: accuracy:0.3333333333333333 loss: 1.8462063074111938
3: accuracy:0.4326241134751773 loss: 1.6321767568588257
4: accuracy:0.49645390070921985 loss: 1.5053471326828003
5: accuracy:0.5106382978723404 loss: 1.2786084413528442
6: accuracy:0.5390070921985816 loss: 1.175156831741333
7: accuracy:0.524822695035461 loss: 1.0732951164245605
8: accuracy:0.5531914893617021 loss: 0.9134953022003174
9: accuracy:0.5815602836879432 loss: 0.7435367107391357
10: accuracy:0.5815602836879432 loss: 0.6175999641418457
11: accuracy:0.5815602836879432 loss: 0.5391039252281189
12: accuracy:0.5815602836879432 loss: 0.4725273847579956
13: accuracy:0.5886524822695035 loss: 0.34439972043037415
14: accuracy:0.6028368794326241 loss: 0.24604764580726624
15:

137: accuracy:0.6666666666666666 loss: 0.0009977295994758606
138: accuracy:0.6666666666666666 loss: 0.00104542076587677
139: accuracy:0.6666666666666666 loss: 0.0009148344397544861
140: accuracy:0.6666666666666666 loss: 0.0008632875978946686
141: accuracy:0.6666666666666666 loss: 0.0010071024298667908
142: accuracy:0.6666666666666666 loss: 0.0009018294513225555
143: accuracy:0.6666666666666666 loss: 0.0010723061859607697
144: accuracy:0.6666666666666666 loss: 0.001063574105501175
145: accuracy:0.6666666666666666 loss: 0.0008732005953788757
146: accuracy:0.6666666666666666 loss: 0.0009153597056865692
147: accuracy:0.6666666666666666 loss: 0.0008699037134647369
148: accuracy:0.6666666666666666 loss: 0.0009642206132411957
149: accuracy:0.6666666666666666 loss: 0.0008136220276355743
150: accuracy:0.6595744680851063 loss: 0.0009157955646514893
151: accuracy:0.6595744680851063 loss: 0.0007381327450275421
152: accuracy:0.6595744680851063 loss: 0.0007743127644062042
153: accuracy:0.65957446808